In [71]:
import pandas as pd
import tabix as tb
import numpy as np

In [72]:
tumor_df = pd.read_csv("../results/cnv_analysis/Tumor.All.txt",header=0,sep="\t")
tumor_bed="../results/cnv_analysis/tumor_cnv.bed"

In [73]:
tumor_df = tumor_df.sort_values(by="Chromosome",ascending=True)

In [74]:
with open(tumor_bed,"w") as f:
    for i,cnv in tumor_df.iterrows():
        if (i <= 30) :
            if (cnv["q-value"] <= 0.05) and (int(cnv["Region Size [bp]"]) > 50):
                f.write(str(int(cnv["Chromosome"])) + "\t" + 
                        str(int(cnv["Region Start [bp]"])) + "\t" +
                        str(int(cnv["Region End [bp]"]))+"\t" +
                        str(int(cnv["q-value"]))+"\t" +
                        str(int(cnv["Aberration Kind"])) +"\n")
f.close()

In [75]:
header=["Chr","start","end","q-value","Abr"]
cnv_bed = pd.read_csv("../results/cnv_analysis/tumor_cnv.bed",sep="\t",names=header)

In [76]:
cnv_bed.shape
cnv_bed

,Chr,start,end,q-value,Abr
0,1,27106855,27127935,0,0
1,1,26648381,26666809,0,0
2,1,27093657,27101201,0,0
3,10,116079327,116168638,0,0
4,10,117244091,117380528,0,0
5,10,116791828,116917867,0,0
6,10,116346514,116438345,0,0
7,10,114777939,114893489,0,0
8,10,118087220,118087298,0,0
9,10,125274134,125275363,0,0


In [77]:
segments = pd.read_csv("../results/cnv_analysis/filtered_samples_cnv.csv",sep="\t",header=0)
segments.head()

,Sample,Chromosome,Start,End,Num_Probes,Segment_Mean
0,TCGA.2F.A9KO.01A,1,3218610,103464007,57847.0,-0.1624
1,TCGA.2F.A9KO.01A,1,103466917,103467218,2.0,-2.9977
2,TCGA.2F.A9KO.01A,1,103467374,107345135,1762.0,-0.1900
3,TCGA.2F.A9KO.01A,1,107350462,107350511,2.0,-1.9026
4,TCGA.2F.A9KO.01A,1,107350931,114237283,4163.0,-0.1855


In [78]:
numb_sample = len(segments["Sample"].unique())
sample_name = segments["Sample"].unique()

In [79]:
cnv_check= pd.DataFrame(np.zeros((cnv_bed.shape[0],numb_sample)),columns=sample_name,dtype=int)
cnv_check.head()

,TCGA.2F.A9KO.01A,TCGA.2F.A9KP.01A,TCGA.2F.A9KQ.01A,TCGA.2F.A9KR.01A,TCGA.2F.A9KT.01A,TCGA.2F.A9KW.01A,TCGA.4Z.AA7M.01A,TCGA.4Z.AA7N.01A,TCGA.4Z.AA7O.01A,TCGA.4Z.AA7Q.01A,...,TCGA.ZF.AA4X.01A,TCGA.ZF.AA51.01A,TCGA.ZF.AA52.01A,TCGA.ZF.AA53.01A,TCGA.ZF.AA54.01A,TCGA.ZF.AA56.01A,TCGA.ZF.AA58.01A,TCGA.ZF.AA5H.01A,TCGA.ZF.AA5N.01A,TCGA.ZF.AA5P.01A
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
for i in range(cnv_bed.shape[0]):
    j = 0
    while j < segments.shape[0]:
        id_name= segments["Sample"][j]
        if cnv_check[id_name][i] == 0:
            if ((segments["Chromosome"][j] == cnv_bed["Chr"][i]) 
                and (segments["Start"][j] <= cnv_bed["start"][i]) 
                and (segments["End"][j] >= cnv_bed["end"][i])):
                cnv_check[id_name][i] += 1
                j +=1
            else: j +=1
        else: j +=1

In [81]:
cnv_check.head()

,TCGA.2F.A9KO.01A,TCGA.2F.A9KP.01A,TCGA.2F.A9KQ.01A,TCGA.2F.A9KR.01A,TCGA.2F.A9KT.01A,TCGA.2F.A9KW.01A,TCGA.4Z.AA7M.01A,TCGA.4Z.AA7N.01A,TCGA.4Z.AA7O.01A,TCGA.4Z.AA7Q.01A,...,TCGA.ZF.AA4X.01A,TCGA.ZF.AA51.01A,TCGA.ZF.AA52.01A,TCGA.ZF.AA53.01A,TCGA.ZF.AA54.01A,TCGA.ZF.AA56.01A,TCGA.ZF.AA58.01A,TCGA.ZF.AA5H.01A,TCGA.ZF.AA5N.01A,TCGA.ZF.AA5P.01A
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [82]:
cnv_check = cnv_check.T

# Machine Learning part

In [83]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
import matplotlib.pyplot as plt

In [84]:
label = pd.read_csv("../results/gene_exp_analysis/clinical.csv",
                    header=0,sep="\t")

In [85]:
label = label[label['sample'].isin(cnv_check.index)]

In [92]:
X = cnv_check
y = label["OS"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [97]:
from umap import UMAP
import plotly.express as px

umap_2d = UMAP(n_components=2, init='random', random_state=0)

proj_2d = umap_2d.fit_transform(cnv_check)

fig_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=label["stage"]
)

fig_2d.show()


In [91]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model=RandomForestClassifier(random_state=42)

# defining parameter range
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [10,12,14,16,20],
    'criterion' :['gini', 'entropy']
}

In [93]:
grid = GridSearchCV(model, param_grid, refit = True, verbose = 1)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
from sklearn.metrics import classification_report, confusion_matrix
grid_predictions = grid.predict(X_test)
print(classification_report(y_test, grid_predictions))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.52      1.00      0.69        65
           1       0.00      0.00      0.00        59

    accuracy                           0.52       124
   macro avg       0.26      0.50      0.34       124
weighted avg       0.27      0.52      0.36       124



Can not predict death 